In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import matplotlib.pyplot as plt
from arch import arch_model
from arch.__future__ import reindexing

In [2]:
df = yf.download('^GSPC',start='1900-03-01', proxy='127.0.0.1:7890')
df.drop(columns=['Volume'],inplace=True)
df['return'] = 100*df['Adj Close'].pct_change()
df['log_return'] = 100*(np.log2(df['Adj Close']) - np.log2(df['Adj Close'].shift(1)))
df.dropna(inplace=True)
df['realized volatility'] = df['return'].rolling(22).std()
df.dropna(inplace=True)
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,return,log_return,realized volatility
Date,,,,,,,,
1928-02-01,17.530001,17.530001,17.530001,17.530001,17.530001,-0.227655,-0.328811,0.746812
1928-02-02,17.629999,17.629999,17.629999,17.629999,17.629999,0.570442,0.820635,0.746972
1928-02-03,17.400000,17.400000,17.400000,17.400000,17.400000,-1.304592,-1.894513,0.794304
1928-02-06,17.450001,17.450001,17.450001,17.450001,17.450001,0.287363,0.413983,0.772743
1928-02-07,17.440001,17.440001,17.440001,17.440001,17.440001,-0.057308,-0.082702,0.758987
...,...,...,...,...,...,...,...,...
2022-03-28,4541.089844,4575.649902,4517.689941,4575.520020,4575.520020,0.714496,1.027134,1.482212
2022-03-29,4602.859863,4637.299805,4589.660156,4631.600098,4631.600098,1.225655,1.757497,1.434337
2022-03-30,4624.200195,4627.770020,4581.319824,4602.450195,4602.450195,-0.629370,-0.910858,1.443096


In [3]:
# gm_result.params['mu'],gm_result.params['omega'],gm_result.params['alpha[1]'],gm_result.params['beta[1]']
def arch_pram(data):
    garch_model = arch_model(data, p = 1, q = 1,
                      mean = 'constant', vol = 'GARCH', dist = 'normal')
    gm_result = garch_model.fit(disp='off')
    return gm_result.params

In [4]:
df['alpha[1]'] = df['return'].rolling(22).apply(lambda x: arch_pram(x)['alpha[1]'])
df['beta[1]'] = df['return'].rolling(22).apply(lambda x: arch_pram(x)['beta[1]'])
df

/Users/shinachuan/miniforge3/envs/tensorflow/lib/python3.9/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.09887. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/shinachuan/miniforge3/envs/tensorflow/lib/python3.9/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.09458. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/shinachuan/mini

,Open,High,Low,Close,Adj Close,return,log_return,realized volatility,alpha[1],beta[1]
Date,,,,,,,,,,
1928-02-01,17.530001,17.530001,17.530001,17.530001,17.530001,-0.227655,-0.328811,0.746812,NaN,NaN
1928-02-02,17.629999,17.629999,17.629999,17.629999,17.629999,0.570442,0.820635,0.746972,NaN,NaN
1928-02-03,17.400000,17.400000,17.400000,17.400000,17.400000,-1.304592,-1.894513,0.794304,NaN,NaN
1928-02-06,17.450001,17.450001,17.450001,17.450001,17.450001,0.287363,0.413983,0.772743,NaN,NaN
1928-02-07,17.440001,17.440001,17.440001,17.440001,17.440001,-0.057308,-0.082702,0.758987,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-03-28,4541.089844,4575.649902,4517.689941,4575.520020,4575.520020,0.714496,1.027134,1.482212,0.000000e+00,0.971109
2022-03-29,4602.859863,4637.299805,4589.660156,4631.600098,4631.600098,1.225655,1.757497,1.434337,1.019311e-13,0.971074
2022-03-30,4624.200195,4627.770020,4581.319824,4602.450195,4602.450195,-0.629370,-0.910858,1.443096,5.266536e-18,0.962702


In [5]:
df.dropna(inplace=True)
df.to_csv('lstm_garch.csv')